<a href="https://colab.research.google.com/github/ZeleOeO/Artificial-Intelligence-Projects/blob/main/Sarcasm_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/learning-datasets/sarcasm.json \
    -O /tmp/sarcasm.json

--2023-03-29 10:04:13--  https://storage.googleapis.com/learning-datasets/sarcasm.json
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.2.128, 142.250.141.128, 2607:f8b0:4023:c0b::80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.2.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5643545 (5.4M) [application/json]
Saving to: ‘/tmp/sarcasm.json’

/tmp/sarcasm.json   100%[===================>]   5.38M  --.-KB/s    in 0.03s   

2023-03-29 10:04:13 (165 MB/s) - ‘/tmp/sarcasm.json’ saved [5643545/5643545]



In [ ]:
vocab_size = 10000
embedding_dim = 16
max_length = 100
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 20000

In [ ]:
f = open("/tmp/sarcasm.json", "r")
datastore = json.load(f)

sentences = []
labels = []
urls = []

for item in datastore:
  sentences.append(item["headline"])
  labels.append(item["is_sarcastic"])
  urls.append(item["article_link"])

print(sentences[:10])

["former versace store clerk sues over secret 'black code' for minority shoppers", "the 'roseanne' revival catches up to our thorny political mood, for better and worse", "mom starting to fear son's web series closest thing she will have to grandchild", 'boehner just wants wife to listen, not come up with alternative debt-reduction ideas', 'j.k. rowling wishes snape happy birthday in the most magical way', "advancing the world's women", 'the fascinating case for eating lab-grown meat', 'this ceo will send your kids to school, if you work for his company', 'top snake handler leaves sinking huckabee campaign', "friday's morning email: inside trump's presser for the ages"]


In [ ]:
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

tokenizer = Tokenizer(oov_token = oov_tok, num_words=vocab_size)

tokenizer.fit_on_texts(sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(sentences)
padded = pad_sequences(sequences, padding = "post")

In [ ]:
print(word_index)
print(padded[0])
print(padded.shape)

In [ ]:
#create training sentence and label and testing sentence and label
training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

In [ ]:
tokenizer.fit_on_texts(training_sentences)
word_index= tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding = padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding =padding_type, truncating=trunc_type)

In [ ]:
import numpy as np
training_sequences = np.array(training_padded)
training_labels = np.array(training_labels)
testing_sequences = np.array(testing_padded)
testing_labels =np.array(testing_labels)
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dense(24, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid"),
])

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
model.fit(training_sequences, training_labels, epochs=30, validation_data=(testing_sequences, testing_labels), verbose = 2)

Epoch 1/30
625/625 - 4s - loss: 0.6645 - accuracy: 0.5954 - val_loss: 0.5842 - val_accuracy: 0.7180 - 4s/epoch - 6ms/step
Epoch 2/30
625/625 - 3s - loss: 0.4396 - accuracy: 0.8306 - val_loss: 0.3880 - val_accuracy: 0.8426 - 3s/epoch - 5ms/step
Epoch 3/30
625/625 - 3s - loss: 0.3176 - accuracy: 0.8725 - val_loss: 0.3539 - val_accuracy: 0.8517 - 3s/epoch - 4ms/step
Epoch 4/30
625/625 - 4s - loss: 0.2629 - accuracy: 0.8975 - val_loss: 0.3411 - val_accuracy: 0.8568 - 4s/epoch - 6ms/step
Epoch 5/30
625/625 - 3s - loss: 0.2276 - accuracy: 0.9119 - val_loss: 0.3402 - val_accuracy: 0.8591 - 3s/epoch - 4ms/step
Epoch 6/30
625/625 - 3s - loss: 0.2003 - accuracy: 0.9232 - val_loss: 0.3628 - val_accuracy: 0.8462 - 3s/epoch - 4ms/step
Epoch 7/30
625/625 - 3s - loss: 0.1783 - accuracy: 0.9331 - val_loss: 0.3622 - val_accuracy: 0.8553 - 3s/epoch - 4ms/step
Epoch 8/30
625/625 - 3s - loss: 0.1600 - accuracy: 0.9413 - val_loss: 0.3683 - val_accuracy: 0.8559 - 3s/epoch - 5ms/step
Epoch 9/30
625/625 - 3s 

In [ ]:
sentence = [
    "God gave men both a penis and a brain, but unfortunately not enough blood supply to run both at the same time.",
    "The sun is out today.",
    "There's so much going on, it's almost like God is punishing us.",
    "When people ask me stupid questions, it is my legal obligation to give a sarcastic remark.",
    "Cows lose their jobs as milk prices drop"
]

sequences = tokenizer.texts_to_sequences(sentence)
padded = pad_sequences(sequences, maxlen=max_length, padding = padding_type, truncating=trunc_type)

for index,i in enumerate(model.predict(padded)):
  if i >= 0.5:
    print("'"+ sentence[index]+"'"+ ": Sarcastic")
  else:
    print("'"+ sentence[index]+"'"+ ": Not sarcastic")

1/1 [==============================] - 0s 24ms/step
'God gave men both a penis and a brain, but unfortunately not enough blood supply to run both at the same time.': Sarcastic
'The sun is out today.': Not sarcastic
'There's so much going on, it's almost like God is punishing us.': Not sarcastic
'When people ask me stupid questions, it is my legal obligation to give a sarcastic remark.': Not sarcastic
'Cows lose their jobs as milk prices drop': Sarcastic
